In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!pip install numpy requests nlpaug
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
# import nlpaug.augmenter.word as naw
import torch
import re
import random

In [ ]:
df = pd.read_excel("/content/gdrive/MyDrive/프로젝트NLP/train_원본.xlsx")

In [ ]:
pd.read_excel("/content/gdrive/MyDrive/프로젝트NLP/real_data.xlsx")

,Customer_Description,y
0,"This customer is 56 years old, works as a hous...",no
1,"The customer is a 56-year-old housemaid, marri...",no
2,"At 56 years old, this customer is employed as ...",no
3,This is a 56-year-old customer working as a ho...,no
4,"A 56-year-old customer, married, working as a ...",no
...,...,...
205935,"This customer is 74 years old, works as a reti...",no
205936,"The customer is a 74-year-old retired, married...",no
205937,"At 74 years old, this customer is employed as ...",no
205938,This is a 74-year-old customer working as a re...,no


In [ ]:
# RoBERTa 기반 증강기 설정 (substitute만 설정)
substitute_aug = naw.ContextualWordEmbsAug(
    model_path='roberta-base',
    action="substitute",
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
# RoBERTa 기반 증강기 설정 (substitute만 설정)
substitute_aug = naw.ContextualWordEmbsAug(
    model_path='roberta-base',
    action="substitute",
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# 숫자를 제외한 텍스트 증강 함수
def augment_text(text):
    # 부호를 포함한 숫자를 추출하도록 정규식 수정
    numbers = re.findall(r'-?\d+\.?\d*', text)  # 음수를 포함한 숫자 찾기
    korean_chars = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ']  # 한글 자음 사용
    number_placeholders = [korean_chars[i] for i in range(len(numbers))]  # 고유한 한글 자음으로 마스킹

    # 숫자를 고유한 한글 자음으로 치환
    for i, num in enumerate(numbers):
        text = text.replace(num, number_placeholders[i], 1)

    # 텍스트 증강 수행 (숫자가 없는 상태에서 substitute만 수행)
    augmented_text = substitute_aug.augment(text)

    # 리스트로 반환된 경우 문자열로 변환
    if isinstance(augmented_text, list):
        augmented_text = ' '.join(augmented_text)

    # 증강된 텍스트에서 고유한 한글 자음을 다시 원래 숫자로 복원
    for i, placeholder in enumerate(number_placeholders):
        if placeholder in augmented_text:
            augmented_text = augmented_text.replace(placeholder, numbers[i], 1)

    # 연속된 공백을 하나의 공백으로 변환
    augmented_text = re.sub(r'\s+', ' ', augmented_text).strip()

    return augmented_text

In [ ]:
df['text_aug'] = df['text'].apply(lambda x: augment_text(x))

In [ ]:
df[['text_aug','y']].to_excel("/content/gdrive/MyDrive/프로젝트NLP/train_aug.xlsx", index = False)

## 숫자 안변했는지 test

In [ ]:
def extract_numbers(text):
    return re.findall(r"\d+\.?\d*", text)

def compare_numbers(num1, num2):
    try:
        return float(num1) == float(num2)
    except ValueError:
        return False

# 숫자 비교 및 다른 값 필터링 함수 (수정)
def find_mismatched_numbers_refined(df1, df2):
    mismatches = []
    for idx, (original_text, aug_text) in enumerate(zip(df1['text'], df2['text_aug'])):
        original_numbers = extract_numbers(original_text)
        aug_numbers = extract_numbers(aug_text)

        if len(original_numbers) != len(aug_numbers):
            mismatches.append((idx, original_numbers, aug_numbers))
        else:
            for o_num, a_num in zip(original_numbers, aug_numbers):
                if not compare_numbers(o_num, a_num):
                    mismatches.append((idx, original_numbers, aug_numbers))
                    break

    return mismatches

In [ ]:
mismatched_sentences = find_mismatched_numbers_refined(df, aug)

In [ ]:
len(mismatched_sentences)

5108

In [ ]:
df['text'][6329]

'This customer is 54 years old, works as a services, and is divorced. They have completed university_degree education, have no credit issues, and do not have a housing or personal loan. The previous contact was made by cellular, with the most recent contact occurring on a tue in may. The customer has never been contacted before and did not participate in previous marketing campaigns. At that time, the quarterly employment change rate was -1.8%, the consumer price index was 92.893, the consumer confidence index was -46.2, the EURIBOR 3-month rate was 1.344, and the employee count was 5099.1.'

In [ ]:
aug['text_aug'][6329]

'This relationship is 54 years old, works as a services, and is divorced. They have completed 3_degree levels, have no credit issues, yet do not have a housing or commercial loan. The previous contact was made by April, with the most recent contact occurring on a tue in may. Single customer has never been contacted before and did not participate in previous marketing campaigns. At that time, the quarterly value change statistic was -1.8%, the consumer price index was 92.893, the consumer confidence index was -46.2, the full 3-month rate was 1.344, and the employee count was 5099.1.'

## 역번역

In [ ]:
df

,text,y
0,"This customer is 40 years old, works as a blue...",yes
1,"This customer is 44 years old, works as a admi...",no
2,"This customer is 35 years old, works as a admi...",yes
3,"This customer is 28 years old, works as a blue...",no
4,"This customer is 52 years old, works as a tech...",no
...,...,...
32945,"This customer is 39 years old, works as a blue...",no
32946,"This customer is 36 years old, works as a tech...",no
32947,"This customer is 28 years old, works as a mana...",no
32948,"This customer is 31 years old, works as a tech...",no


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# MarianMT 모델 불러오기 (예: 영어 -> 독일어 -> 영어)
model_name_en_to_de = 'Helsinki-NLP/opus-mt-en-de'
model_name_de_to_en = 'Helsinki-NLP/opus-mt-de-en'

tokenizer_en_to_de = MarianTokenizer.from_pretrained(model_name_en_to_de)
model_en_to_de = MarianMTModel.from_pretrained(model_name_en_to_de)

tokenizer_de_to_en = MarianTokenizer.from_pretrained(model_name_de_to_en)
model_de_to_en = MarianMTModel.from_pretrained(model_name_de_to_en)

def batch_back_translate(texts):
    # 여러 문장을 입력으로 받아 한 번에 처리하는 함수
    batch_inputs = tokenizer_en_to_de(texts, return_tensors="pt", padding=True, truncation=True, max_length=160)
    translated_tokens = model_en_to_de.generate(**batch_inputs)
    translated_texts = [tokenizer_en_to_de.decode(t, skip_special_tokens=True) for t in translated_tokens]

    back_batch_inputs = tokenizer_de_to_en(translated_texts, return_tensors="pt", padding=True, truncation=True, max_length=160)
    back_translated_tokens = model_de_to_en.generate(**back_batch_inputs)
    back_translated_texts = [tokenizer_de_to_en.decode(t, skip_special_tokens=True) for t in back_translated_tokens]

    return back_translated_texts

# 배치 크기 설정
batch_size = 32
batch_results  = []
for i in range(0, len(df['text']), batch_size):
    batch_texts = df['text'][i:i+batch_size].tolist()
    batch_results.extend(batch_back_translate(batch_texts))

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
df['test_BT'] = batch_results
df[['test_BT','y']].to_excel("/content/gdrive/MyDrive/프로젝트NLP/train_BT.xlsx", index = False)

'This customer is 33 years old, works as a management, and is married. They have completed university_degree education, have no credit issues, and do not have a housing or personal loan. The previous contact was made by cellular, with the most recent contact occurring on a tue in nov. The customer has never been contacted before and did not participate in previous marketing campaigns. At that time, the quarterly employment change rate was -0.1%, the consumer price index was 93.2, the consumer confidence index was -42.0, the EURIBOR 3-month rate was 4.1530000000000005, and the employee count was 5195.8.'

In [ ]:
batch_results[5]

'This customer is 33 years old, works as a management and is married. They have completed university_degree education, have no credit problems, and have no apartment or personal loan. The previous contact was made by cellular, with the latest contact on a tu in nov. The customer has never been contacted before and did not participate in previous marketing campaigns. At that time, the quarterly employment rate was -0.1%, the consumer price index was 93.2, the consumer confidence index was -42.0, the EURIBOR 3 month rate was 4,1530000000000005, and the number of employees was 5195.8.'